starting the eva server

In [37]:
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

[  -z "$(lsof -ti:5432)" ] || kill -9 "$(lsof -ti:5432)"
nohup eva_server > eva.log 2>&1 &
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF FastRCNNObjectDetector already exists, nothing added.
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF MVITActionRecognition already exists, nothing added.
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Array_Count already exists, nothing added.
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Crop already exists, nothing added.
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF DummyObjectDetector already exists, nothing added.
12-19-2022 01:04:36 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF DummyMultiObjectDetector already exists, nothing added.
12-19-2022 01:04:36 CRITICAL[server:server.py:start_server:0096] Start Server
12-19-2022 01:04:36 CR

sh: line 0: kill: 797
808: arguments must be process or job IDs


Note: you may need to restart the kernel to use updated packages.


Loadind the file and droping the data instance if it exists and creating new one

In [39]:
cursor.execute("DROP TABLE MRI;")
response = cursor.fetch_all()
print(response)
cursor.execute("LOAD FILE 'test.mp4' INTO MRI;")
response = cursor.fetch_all()
print(response)
cursor.execute("""SELECT id FROM MRI WHERE id < 5""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                  0
0  Table Successfully dropped: MRI
@query_time: 0.0074299170000244885
@status: ResponseStatus.SUCCESS
@batch: 
                                                 0
0  Video successfully added at location: test.mp4
@query_time: 0.015456292000010308
@status: ResponseStatus.SUCCESS
@batch: 
    mri.id
0       0
1       1
2       2
3       3
4       4
@query_time: 0.025677874999985306


Creating the UDF method with input and output parameters and specifying the implementation path

In [40]:
cursor.execute("""DROP UDF MRICNN;""")
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS 
                  MRICNN
                  INPUT (data NDARRAY UINT8(3, 224, 224)) 
                  OUTPUT (label TEXT(2)) 
                  TYPE  Classification 
                  IMPL 'UDF.py';
        """)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                  0
0  UDF MRICNN successfully dropped
@query_time: 0.0059727910000333395
@status: ResponseStatus.SUCCESS
@batch: 
                                                 0
0  UDF MRICNN successfully added to the database.
@query_time: 0.225746499999957


Performing the prediction querry

In [50]:
cursor.execute("""SELECT id,data, MRICNN(data)
                  FROM MRI
                  WHERE id < 5
                  """)
response = cursor.fetch_all()
print(response.batch)

   mri.id  \
0       0   
1       1   
2       2   
3       3   
4       4   

                                                                                              mri.data  \
0  [[[2 2 2]\n [2 2 2]\n [2 2 2]\n ...\n [2 2 2]\n [2 2 2]\n [2 2 2]], [[2 2 2]\n [2 2 2]\n [2 2 2]...   
1  [[[2 2 2]\n [2 2 2]\n [2 2 2]\n ...\n [2 2 2]\n [2 2 2]\n [2 2 2]], [[2 2 2]\n [2 2 2]\n [2 2 2]...   
2  [[[2 2 2]\n [2 2 2]\n [2 2 2]\n ...\n [2 2 2]\n [2 2 2]\n [2 2 2]], [[2 2 2]\n [2 2 2]\n [2 2 2]...   
3  [[[2 2 2]\n [2 2 2]\n [2 2 2]\n ...\n [2 2 2]\n [2 2 2]\n [2 2 2]], [[2 2 2]\n [2 2 2]\n [2 2 2]...   
4  [[[2 2 2]\n [2 2 2]\n [2 2 2]\n ...\n [2 2 2]\n [2 2 2]\n [2 2 2]], [[2 2 2]\n [2 2 2]\n [2 2 2]...   

  mricnn.label  
0            1  
1            1  
2            1  
3            1  
4            1  
